In [1]:
import pandas as pd

data = pd.read_csv('data/GA_train.csv')
data.head()

,id_no,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,1,325,5.05,385,0,4463.92,599.974,21.560558,15.770406,28.373427,0.18927
1,2,325,5.05,385,0,4463.92,599.974,21.560558,15.770406,28.373427,0.18927
2,3,325,4.76,385,0,4463.92,599.974,21.560558,15.770406,28.373427,0.18927
3,4,325,4.95,385,0,4463.92,599.974,21.560558,15.770406,28.373427,0.18927
4,5,325,4.86,385,0,4463.92,599.974,21.560558,15.770406,28.373427,0.18927


In [10]:
from sdv.tabular import CTGAN

model = CTGAN(primary_key='id_no',log_frequency=False, cuda=True, epochs=500, batch_size=10, generator_lr=.000007, discriminator_lr=.000009)
model.fit(data)

In [11]:
new_data = model.sample(50)

In [12]:
new_data.to_csv('data/GA_synth/GAsynth7_012822.csv')
new_data.head()

,id_no,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,0,341,6.03,366,390,5320.75,1323.896,20.581257,16.120036,26.428849,0.191288
1,1,303,6.03,349,205,4509.10,1323.896,21.114204,15.692693,27.253351,0.165507
2,2,226,4.07,460,274,5257.76,730.235,20.581257,16.934281,27.702107,0.180112
3,3,302,5.16,655,70,4433.11,886.612,22.450957,15.692693,28.061688,0.191288
4,4,341,3.57,315,294,5320.75,753.444,21.958849,15.692693,28.394983,0.130817


In [13]:
# get aggregate data
targetDataLoc = 'data/GA_test.csv'
aggDataLoc = 'data/GA_synth/GAsynth7_012822.csv'

aggDf = pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)
#aggDf.head()
targetDf.head()

,id_no,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,1,340,3.35,422,0,3545.32,413.282,18.354044,12.543772,24.636933,0.129465
1,2,340,2.99,422,0,3545.32,413.282,18.354044,12.543772,24.636933,0.129465
2,3,340,3.08,422,0,3545.32,413.282,18.354044,12.543772,24.636933,0.129465
3,4,340,3.10,422,0,3545.32,413.282,18.354044,12.543772,24.636933,0.129465
4,5,340,3.13,422,0,3545.32,413.282,18.354044,12.543772,24.636933,0.129465


In [14]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [15]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Yield (tons/acre)"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = xDf.reset_index(drop=True)
test_yDf = yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [16]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["mae"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10,20,40,80],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['mae'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [(RandomForestRegressor(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              (SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")],
             (LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             (BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             (DecisionTreeRegressor(), dtParamGrid, "dtTup.pkl")]

N = 20
workingDir = 'working_dir'
numFeatures = 8 # 11

In [17]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  rfTup
Avg MAE:  1969.073
Avg R squared:  -0.841
Best MAE:  947.067
Best R squared:  0.779
Parameters of the best model:  {'model__criterion': 'mae', 'model__max_depth': 15, 'model__n_estimators': 50}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
0.418236
0.8286084465023642
model:  knnTup
Avg MAE:  2029.77
Avg R squared:  -1.235
Best MAE:  936.352
Best R squared:  0.771
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 5, 'model__weights': 'distance'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
0.028090570218439092
0.986431035981289
model:  s